### ShakesPearean Text

#### Creating a Dataset

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential, callbacks
import numpy as np
import requests

In [2]:
r = requests.get("https://homl.info/shakespeare")
open("shakespearean_text.txt", "wb").write(r.content)

1115394

In [3]:
with open("shakespearean_text.txt") as f:
    shakespear_text = f.read()

In [4]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespear_text])

In [5]:
tokenizer.texts_to_sequences(["Hello"])

[[7, 2, 12, 12, 4]]

In [6]:
tokenizer.sequences_to_texts([[7, 2, 12, 12, 4]])

['h e l l o']

In [7]:
max_id = len(tokenizer.word_index)
# dataset_size = tokenizer.document_count

In [8]:
max_id

39

In [9]:
[encoded_text] = np.array(tokenizer.texts_to_sequences([shakespear_text])) - 1

In [10]:
encoded_text.shape

(1115394,)

In [11]:
dataset_size = len(encoded_text)
dataset_size

1115394

#### Spliting Dataset

In [12]:
train_size = int(dataset_size * 0.9)
train_set = tf.data.Dataset.from_tensor_slices(encoded_text[:train_size])
validation_set = tf.data.Dataset.from_tensor_slices(encoded_text[train_size:])

#### Batching dataset

In [13]:
n_steps = 100
window_length = n_steps + 1
train_set = train_set.window(window_length, shift=1, drop_remainder=True)

In [14]:
for i in train_set.take(1):
    for x in i.as_numpy_iterator():
        print(x, end=" ")

19 5 8 7 2 0 18 5 2 5 35 1 9 23 10 21 1 19 3 8 1 0 16 1 0 22 8 3 18 1 1 12 0 4 9 15 0 19 13 8 2 6 1 8 17 0 6 1 4 8 0 14 1 0 7 22 1 4 24 26 10 10 4 11 11 23 10 7 22 1 4 24 17 0 7 22 1 4 24 26 10 10 19 5 8 7 2 0 18 5 2 5 35 1 9 23 10 15 3 13 0 

In [15]:
train_set = train_set.flat_map(lambda window: window.batch(window_length))

In [16]:
for i in train_set.take(1):
    print(i)

tf.Tensor(
[19  5  8  7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1
  0 22  8  3 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1
  4  8  0 14  1  0  7 22  1  4 24 26 10 10  4 11 11 23 10  7 22  1  4 24
 17  0  7 22  1  4 24 26 10 10 19  5  8  7  2  0 18  5  2  5 35  1  9 23
 10 15  3 13  0], shape=(101,), dtype=int64)


In [17]:
batch_size = 32
train_set = train_set.shuffle(10000).batch(batch_size)
train_set = train_set.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [18]:
train_set = train_set.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
)
train_set = train_set.prefetch(tf.data.AUTOTUNE)

#### Training Char-RNN

In [ ]:
model1 = Sequential([
    layers.GRU(128, input_shape=[None, max_id],
               dropout=0.2,
            #    recurrent_dropout=0.2, 
               return_sequences=True), 
    layers.GRU(128, dropout=0.2, 
            #    recurrent_dropout=0.2, 
               return_sequences=True), 
    layers.TimeDistributed(layers.Dense(max_id, activation="softmax"))
])
model1.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history1 = model1.fit(train_set, epochs=10)

In [ ]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts))
    return tf.one_hot(X, max_id)

In [ ]:
X_new = preprocess(["How are yo"])
Y_pred = model1.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

In [19]:
def next_char(text, temperature):
    X_new = preprocess([text])
    y_proba = model1.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [20]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

#### Statefull RNN

In [23]:
dataset = tf.data.Dataset.from_tensor_slices(encoded_text[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [28]:
model2 = Sequential([
        layers.GRU(128, return_sequences=True, stateful=True, 
                dropout=0.2, 
                # recurrent_dropout=0.2, 
                batch_input_shape=[1, None, max_id]), 
        layers.GRU(128, return_sequences=True, stateful=True, 
                dropout=0.2, 
                # recurrent_dropout=0.2,
                ), 
        layers.TimeDistributed(layers.Dense(max_id, activation="softmax"))
])

In [29]:
class ResetStateCallback(callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [ ]:
model2.compile(loss="sparse_categorical_crossentropy", 
              optimizer="adam")
model2.fit(dataset, epochs=50, callbacks=[ResetStateCallback()])

### Sentiment Analysis